# Analiza czerniaka za pomocą fraktalnej sieci neuronowej

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Zapisujemy konfigurację do zmiennych.

In [2]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

Definiujemy warstwę, która będzie tworzyła obraz z fraktalnych cech podanych jej obrazów.

In [3]:
class Fractal2D(tf.keras.layers.Layer):
    def __init__(self, kernel_size_range):
        super(Fractal2D, self).__init__()
        self.kernel_size_range = kernel_size_range

    def call(self, image_batch):
        for kernel_size in range(self.kernel_size_range[0], self.kernel_size_range[1] + 1, 2):
            split_image_batch = tf.image.extract_patches(image_batch,
                                     sizes=(1, kernel_size, kernel_size, 1),
                                     strides=(1, 1, 1, 1),
                                     rates=(1, 1, 1, 1),
                                     padding='SAME')
            batch_size, row_size, col_size, depth = split_image_batch.shape
            split_image_batch = tf.reshape(split_image_batch, shape=(batch_size, row_size * col_size, kernel_size, kernel_size, 3))
            print(f'kernel_size: {kernel_size} -- shape: {split_image_batch.shape}')
        return image_batch

Ładujemy dane do trenowania i walidacji.

In [4]:
datagen = ImageDataGenerator(validation_split=0.2)
training_set = datagen.flow_from_directory('/small-data',
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical',
                                           subset='training')
validation_set = datagen.flow_from_directory('/small-data',
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical',
                                             subset='validation')

Found 640 images belonging to 3 classes.
Found 159 images belonging to 3 classes.


Zapisujemy ilość rozpoznawalnych diagnoz.

In [5]:
DIAGNOSIS_NUMBER = len(training_set.class_indices)

Tworzymy model, który wykorzystuje wcześniej zdefiniowaną warstwę.

In [6]:
model = tf.keras.Sequential([
    Fractal2D(kernel_size_range=(3, 41)),
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", output_shape=[1280],
                   trainable=False),
    tf.keras.layers.Dense(DIAGNOSIS_NUMBER, activation='softmax')
])

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(training_set, validation_data=validation_set, epochs=20)

shape: (50176, 3, 3, 3)
shape: (50176, 5, 5, 3)
shape: (50176, 7, 7, 3)
shape: (50176, 9, 9, 3)
shape: (50176, 11, 11, 3)
shape: (50176, 13, 13, 3)
shape: (50176, 15, 15, 3)
shape: (50176, 17, 17, 3)
shape: (50176, 19, 19, 3)
shape: (50176, 21, 21, 3)
shape: (50176, 23, 23, 3)
shape: (50176, 25, 25, 3)
shape: (50176, 27, 27, 3)
shape: (50176, 29, 29, 3)
shape: (50176, 31, 31, 3)
shape: (50176, 33, 33, 3)
shape: (50176, 35, 35, 3)
shape: (50176, 37, 37, 3)
shape: (50176, 39, 39, 3)
shape: (50176, 41, 41, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (50176, 3, 3, 3)
shape: (

ResourceExhaustedError: OOM when allocating tensor with shape[32,224,224,363] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ExtractImagePatches]